In [1]:
from osgeo import gdal, ogr
import geopandas as gpd
import os

# Chemin pour le fichier de masque en sortie
output_mask_path = "../results/data/img_pretraitees/masque_foret.tif"

# Chemin pour le fichier vecteur BD Forêt
shp_path = "../../data/project"
shp_name = os.path.join(shp_path, "FORMATION_VEGETALE.shp")

# Chemin pour le fichier shapefile emprise d'étude
emprise_shp_path = os.path.join(shp_path, "emprise_etude.shp")

In [2]:
# Charger la couche vecteur BD Forêt
vector = gpd.read_file(shp_name)

# Charger la couche vecteur "emprise_etude.shp" pour récupérer son emprise
emprise = gpd.read_file(emprise_shp_path)

In [3]:
# Extraire l'emprise de "emprise_etude.shp" (min, max x, min, max y)
x_min, y_min, x_max, y_max = emprise.total_bounds

# Définir les valeurs de TFV à exclure
excluded_tfv = ['FO0', 'FO1', 'FO2', 'FO3', 'LA4', 'LA6']
excluded_tfv_str = "', '".join(excluded_tfv)

# Filtrer les polygones dans la couche BD Forêt (en excluant les TFV)
vector = vector[~vector['CODE_TFV'].isin(excluded_tfv)]  # Appliquer le filtre TFV à exclure

# Définir la taille des pixels (par exemple 30 mètres)
pixel_size = 10  # En mètres

# Calculer les dimensions du raster
cols = int(((x_max - x_min) / pixel_size)-1)
rows = int((y_max - y_min) / pixel_size)

# Créer un raster de sortie avec GDAL
driver = gdal.GetDriverByName("GTiff")
mask_ds = driver.Create(output_mask_path, cols, rows, 1, gdal.GDT_Byte)

print(cols,rows)

10862 7380


/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [4]:
# Définir la géotransformation et la projection du raster
transform = (x_min, pixel_size, 0, y_max, 0, -pixel_size)  # Géotransformation
mask_ds.SetGeoTransform(transform)

# Définir la projection en utilisant l'emprise (projection WKT)
spatial_ref = emprise.crs.to_wkt()  # Récupérer la projection de l'emprise en WKT
mask_ds.SetProjection(spatial_ref)

# Initialiser la bande du raster et la remplir de 0 (valeur par défaut : à exclure)
mask_band = mask_ds.GetRasterBand(1)
mask_band.Fill(0)  # Par défaut, exclure tout
mask_band.SetNoDataValue(0)

# Créer une couche vectorielle temporaire pour stocker les polygones
driver_ogr = ogr.GetDriverByName('Memory')  # Utiliser un driver mémoire pour créer la couche
layer_name = 'temp_layer'
memory_ds = driver_ogr.CreateDataSource('temp')
layer = memory_ds.CreateLayer(layer_name, geom_type=ogr.wkbPolygon)

# Ajouter un champ attributaire pour le code TFV
layer.CreateField(ogr.FieldDefn('CODE_TFV', ogr.OFTString))

# Ajouter les polygones filtrés dans la couche mémoire
for _, row in vector.iterrows():
    geometry = row['geometry']
    
    # Vérifier si la géométrie est valide
    if geometry.is_valid:
        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetGeometry(ogr.CreateGeometryFromWkb(geometry.wkb))
        feature.SetField('CODE_TFV', row['CODE_TFV'])
        layer.CreateFeature(feature)

# Rasteriser la couche mémoire
gdal.RasterizeLayer(mask_ds, [1], layer, burn_values=[1])  # 1 = à classer

# Nettoyer la cache et fermer les objets
mask_band.FlushCache()
mask_ds = None
memory_ds = None  # Supprimer la couche mémoire
print(f"Masque créé avec l'emprise de 'emprise_etude.shp' : {output_mask_path}")

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:8580: RuntimeWarning: Failed to fetch spatial reference on layer temp_layer to build transformer, assuming matching coordinate systems.
  return _gdal.RasterizeLayer(*args, **kwargs)


Masque créé avec l'emprise de 'emprise_etude.shp' : ../results/data/img_pretraitees/masque_foret.tif
